In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import numpy as np
import warnings
from sklearn.preprocessing import LabelEncoder 
from sklearn.model_selection import cross_val_score,train_test_split
import re, string, unicodedata
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from html.parser import HTMLParser
html_parser = HTMLParser()

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
df_pos=pd.read_csv("/content/gdrive/MyDrive/seng607/test_negative_roi/Redmine_Pos.csv")


In [ ]:
df_pos.head()

,Unnamed: 0,id1,Description1,id2,Description2,Type of dependency
0,0,34556,"Currently, the ""Watchers"" section in the new i...",8562,We face the problem that we have many proje...,relates
1,1,30459,"In Redmine 3.4 and earlier, you could update...",34549,Add keyboard shortcuts for the following butto...,blocks
2,2,34495,"""Two-factor authentication"" on a new user form...",1237,Please add support for a one time password ...,relates
3,3,34340,All projects are displayed in the tracker edit...,6938,I have a few ideas to enhance the edit trac...,relates
4,4,33383,jQuery 2 is end of life and we should update ...,34337,#33383#note-3:\r\n> Because the update can bre...,relates


In [ ]:
df_pos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5224 entries, 0 to 5223
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          5224 non-null   int64 
 1   id1                 5224 non-null   int64 
 2   Description1        5224 non-null   object
 3   id2                 5224 non-null   int64 
 4   Description2        5224 non-null   object
 5   Type of dependency  5224 non-null   object
dtypes: int64(3), object(3)
memory usage: 245.0+ KB


In [ ]:
df_neg=pd.read_csv("/content/gdrive/MyDrive/seng607/test_negative_roi/Redmine_Neg.csv")

In [ ]:
df_neg.head()

,Unnamed: 0,id1,Description1,id2,Description2,Type of dependency
0,0,21950,It would be highly useful to those integrat...,4280,"Hey,\r\n\r\nWe have been using Redmine for ...",independent
1,1,683,Currently we are able to produce some very ...,414,It would be great to have a feature that ca...,independent
2,2,532,I would like for members of my integration ...,22001,I want to export structured PDF´s from wiki...,independent
3,3,27381,Would like a hybrid option of % Complete dri...,24151,We use redmine extensively for IT support re...,independent
4,4,4672,"Currently, when copying a project that cont...",270,How about the ability to add/remove modules...,independent


In [ ]:
df_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49903 entries, 0 to 49902
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          49903 non-null  int64 
 1   id1                 49903 non-null  int64 
 2   Description1        49903 non-null  object
 3   id2                 49903 non-null  int64 
 4   Description2        49903 non-null  object
 5   Type of dependency  49903 non-null  object
dtypes: int64(3), object(3)
memory usage: 2.3+ MB


In [ ]:
def processData(df_pos):

    for i in range(len(df_pos)):
          sample=df_pos.iloc[i].Description1
          # print("req1",sample,'\n')
          sample =  re.sub(r"http\S+", "", sample)
          sample = re.sub(r"[\[\],@\'?\.$%_:()\-\"&;<>{}|+!*#]", " ", sample, flags=re.I)
          sample = re.sub(r"\s+[a-zA-Z]\s+", " ", sample)
          sample = ' '.join(w for w in sample.split() if not any(x.isdigit() for x in w))
          sample =sample.lower()
          words=sample
          words = words.split() 
          noise_free_words = [word for word in words if not word.isdigit()]
          noise_free_text = " ".join(noise_free_words)
          df_pos.at[i, 'Description1']= noise_free_text
          # end of req1
          sample2=df_pos.iloc[i].Description2
          # print("req2",sample,"\n")
          sample2 =  re.sub(r"http\S+", "", sample2)
          sample2 = re.sub(r"[\[\],@\'?\.$%_:()\-\"&;<>{}|+!*#]", " ", sample2, flags=re.I)
          sample2 = re.sub(r"\s+[a-zA-Z]\s+", " ", sample2)
          sample2 = ' '.join(w for w in sample2.split() if not any(x.isdigit() for x in w))
          sample2 =sample2.lower()
          words=sample2
          words = words.split() 
          noise_free_words = [word for word in words if not word.isdigit()]
          noise_free_text = " ".join(noise_free_words)
          df_pos.at[i, 'Description2']= noise_free_text          
    return df_pos

In [ ]:
df_firefox_pos=processData(df_pos)

In [ ]:
df_firefox_neg=processData(df_neg)

In [ ]:
df_firefox_pos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5224 entries, 0 to 5223
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          5224 non-null   int64 
 1   id1                 5224 non-null   int64 
 2   Description1        5224 non-null   object
 3   id2                 5224 non-null   int64 
 4   Description2        5224 non-null   object
 5   Type of dependency  5224 non-null   object
dtypes: int64(3), object(3)
memory usage: 245.0+ KB


In [ ]:
df_firefox_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49903 entries, 0 to 49902
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          49903 non-null  int64 
 1   id1                 49903 non-null  int64 
 2   Description1        49903 non-null  object
 3   id2                 49903 non-null  int64 
 4   Description2        49903 non-null  object
 5   Type of dependency  49903 non-null  object
dtypes: int64(3), object(3)
memory usage: 2.3+ MB


In [ ]:
df_firefox_pos.head()

,Unnamed: 0,id1,Description1,id2,Description2,Type of dependency
0,0,34556,currently the watchers section in the new,8562,we face the problem that we have many project,relates
1,1,30459,in redmine and earlier you could update,34549,add keyboard shortcuts for the following butto,blocks
2,2,34495,two factor authentication on new user form,1237,please add support for one time password ser,relates
3,3,34340,all projects are displayed in the tracker edit,6938,have few ideas to enhance the edit tracker,relates
4,4,33383,jquery is end of life and we should update,34337,note \r\n because the update can bre,relates


In [ ]:
df_firefox_neg.head()

,Unnamed: 0,id1,Description1,id2,Description2,Type of dependency
0,0,21950,it would be highly useful to those integrating,4280,hey \r\n\r\nwe have been using redmine for our,independent
1,1,683,currently we are able to produce some very nic,414,it would be great to have feature that can,independent
2,2,532,would like for members of my integration pro,22001,want to export structured pdf´s from wiki,independent
3,3,27381,would like hybrid option of complete drive,24151,we use redmine extensively for it support requ,independent
4,4,4672,currently when copying project that contain,270,how about the ability to add/remove modules on,independent


In [ ]:
print(df_firefox_pos.shape)
print(df_firefox_pos.isnull().sum())

(5224, 6)
Unnamed: 0            0
id1                   0
Description1          0
id2                   0
Description2          0
Type of dependency    0
dtype: int64


In [ ]:
df_firefox_pos.dropna(inplace=True)
df_firefox_pos = df_firefox_pos.reset_index(drop=True)

In [ ]:
print(df_firefox_neg.shape)
print(df_firefox_neg.isnull().sum())

(49903, 6)
Unnamed: 0            0
id1                   0
Description1          0
id2                   0
Description2          0
Type of dependency    0
dtype: int64


In [ ]:
df_firefox_neg.dropna(inplace=True)
df_firefox_neg = df_firefox_neg.reset_index(drop=True)

In [ ]:
print(df_firefox_pos['Type of dependency'].value_counts())

relates       2824
duplicates    2326
blocks          36
copied_to       20
precedes        18
Name: Type of dependency, dtype: int64


In [ ]:
# df_firefox_pos.loc[df_firefox_pos['Type of dependency'] == 'relates']

,Unnamed: 0,id1,Description1,id2,Description2,Type of dependency
0,0,87457,this patch introduces the requirement to symfo,93197,right now only the property var annotation ca,relates
1,1,75043,please use gz instead of gzip for statically,93182,make the compressed file extension configurabl,relates
2,2,93170,i would suggest to deprecate the following pag,93171,introduce fallback site configuration to be,relates
3,3,93170,i would suggest to deprecate the following pag,93171,introduce fallback site configuration to be,relates
4,4,75266,there should be possibility to have at least,92903,it would be great to be able to arrange flexfo,relates
...,...,...,...,...,...,...
1814,1814,3619,there should be something like system policy,890,the order of running the advices in an advice,relates
1816,1816,5660,now finally the property mapper needs to suppo,342,the mvc framework needs to support file / mult,relates
1817,1817,6008,when resource object is discarded the relat,342,the mvc framework needs to support file / mult,relates
1818,1818,10680,there is now use case for content negotiatio,315,the request object mainly webrequest should,relates


In [ ]:
def balancedClasses(df):
    stats = df["label"].value_counts()
    print(stats)
    df_sampledData = pd.DataFrame(columns=df.columns)
    sampleSize = int(input("enter the size"))
    for key in stats.keys():
        allData = df[df["label"] == key]
        if sampleSize > len(allData):
            sampleSize = len(allData)
            df_sample = allData.sample(sampleSize)
        else:
            df_sample = allData.sample(sampleSize)

        df_sampledData = pd.concat([df_sampledData, df_sample], axis=0)  # Add sampled values into the Test Set
    return df_sampledData

In [ ]:
def combineData(dfPos,dfNeg):

    selectTYpes = ['relates']


    dfPos = dfPos[dfPos['Type of dependency'].isin(selectTYpes)]
    print(dfPos['Type of dependency'].value_counts(), len(dfPos))
    # print(dfPos.dependency.unique())
    # input("hit enter")

    df = pd.concat([dfPos, dfNeg])
    df['label'] = df['Type of dependency']
    print(df['Type of dependency'].value_counts(), len(df))
    c = df.label.astype('category')
    d = dict(enumerate(c.cat.categories))
    print(d)
    # {0: 'english', 1: 'spanish'}
    df['label'] = df.label.astype('category').cat.codes
    print(df.head(), df.columns)

    df = balancedClasses(df)
    print(df['Type of dependency'].value_counts(), len(df))

    return df

In [ ]:
df_concat= combineData(df_firefox_pos,df_firefox_neg)

relates    2824
Name: Type of dependency, dtype: int64 2824
independent    49903
relates         2824
Name: Type of dependency, dtype: int64 52727
{0: 'independent', 1: 'relates'}
   Unnamed: 0    id1  ... Type of dependency  label
0           0  34556  ...            relates      1
2           2  34495  ...            relates      1
3           3  34340  ...            relates      1
4           4  33383  ...            relates      1
5           5  34258  ...            relates      1

[5 rows x 7 columns] Index(['Unnamed: 0', 'id1', 'Description1', 'id2', 'Description2',
       'Type of dependency', 'label'],
      dtype='object')
0    49903
1     2824
Name: label, dtype: int64
enter the size2824
independent    2824
relates        2824
Name: Type of dependency, dtype: int64 5648


In [ ]:
df_concat.head()

,Unnamed: 0,id1,Description1,id2,Description2,Type of dependency,label
12544,12544,1240,it will be good to have possibility to change,14582,create project and add all the issues in,independent,0
42208,42208,10459,sorry have posted this in the forum and real,22958,in other words once the macro call executes,independent,0
30134,30134,3487,is it possible to place downloadable files on,31521,at the moment there is no way to add custom va,independent,0
16258,16258,10188,the ability to add new custom informational,2901,my company project budget is based upon the,independent,0
31931,31931,31404,this feature could help the project management,27314,at the moment if you click on the administrat,independent,0


In [ ]:
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5648 entries, 12544 to 178
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          5648 non-null   object
 1   id1                 5648 non-null   object
 2   Description1        5648 non-null   object
 3   id2                 5648 non-null   object
 4   Description2        5648 non-null   object
 5   Type of dependency  5648 non-null   object
 6   label               5648 non-null   object
dtypes: object(7)
memory usage: 353.0+ KB


In [ ]:
print(df_concat['Type of dependency'].value_counts())

independent    2824
relates        2824
Name: Type of dependency, dtype: int64


In [ ]:
# df_firefox_pos.to_csv("/content/gdrive/MyDrive/New_SENG607/RedMine/Redmine_Pos_pocessed.csv",index=False)


In [ ]:
def FirefoxTrainTestSplit(df_firefox):
    # df_firefox = pd.read_csv(inputFile)
    #df_firefox = df_firefox[(df_firefox['req1Product'] == 'Firefox') & (df_firefox['req2Product'] == 'Firefox')]

    #encode labels
    # label encoding the data 
    # le = LabelEncoder() 
    # ### encoding 1: requires, 0: independent
    # df_firefox['label']= le.fit_transform(df_firefox['label'])
    # print("Data in hand\n", df_firefox['label'].value_counts())

    # #get half of -ve samples from inverse of Requires
    # df_neg = getNeg(df_firefox[df_firefox['label']==1])
    

    # df_firefox_1 =  df_firefox[df_firefox['label']==1]
    # df_firefox_0 =  df_firefox[df_firefox['label']==0]
    # #df_firefox_0 = df_firefox_0.sample(len(df_firefox_1))

    # #pump pure black n white a bit
    # lengthIs = len(df_firefox_1)
    # df_dummy = pd.concat([df_neg.sample(int(lengthIs*0)),df_firefox_0.sample(int(lengthIs*1))])
    

    # df_firefox = pd.concat([df_dummy, df_firefox_1])
    # print("extract balanced\n", df_firefox['label'].value_counts())
     
    # ######Shuffle##############
    # df_firefox = df_firefox.sample(frac=1).reset_index(drop=True)
    ###########################

    df_train, df_test = train_test_split(df_firefox, test_size=.2, stratify=df_firefox['label'])
    print("train set\n", df_train['label'].value_counts())
    print("test set\n", df_test['label'].value_counts())

    # df_train.to_csv("/content/gdrive/MyDrive/seng607/test_negative_roi/Typo3_train.csv")
    # df_test.to_csv("/content/gdrive/MyDrive/seng607/test_negative_roi/Typo3_test.csv")
    return df_train,df_test

In [ ]:
df_train_typo,df_test_typo=FirefoxTrainTestSplit(df_concat)

train set
 1    2259
0    2259
Name: label, dtype: int64
test set
 1    565
0    565
Name: label, dtype: int64


In [ ]:
# df1=pd.read_csv("/content/gdrive/MyDrive/seng607/test_negative_roi/Typo3_train.csv")

In [ ]:
df_train_typo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4518 entries, 4357 to 22851
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          4518 non-null   object
 1   id1                 4518 non-null   object
 2   Description1        4518 non-null   object
 3   id2                 4518 non-null   object
 4   Description2        4518 non-null   object
 5   Type of dependency  4518 non-null   object
 6   label               4518 non-null   object
dtypes: object(7)
memory usage: 282.4+ KB


In [ ]:
df_test_typo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1130 entries, 29418 to 36787
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          1130 non-null   object
 1   id1                 1130 non-null   object
 2   Description1        1130 non-null   object
 3   id2                 1130 non-null   object
 4   Description2        1130 non-null   object
 5   Type of dependency  1130 non-null   object
 6   label               1130 non-null   object
dtypes: object(7)
memory usage: 70.6+ KB


In [ ]:
df_train_typo= df_train_typo.rename({'label': 'Label', 'Description1':'req1','Description2':'req2'}, axis=1)

In [ ]:
df_train_typo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4518 entries, 4357 to 22851
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          4518 non-null   object
 1   id1                 4518 non-null   object
 2   req1                4518 non-null   object
 3   id2                 4518 non-null   object
 4   req2                4518 non-null   object
 5   Type of dependency  4518 non-null   object
 6   Label               4518 non-null   object
dtypes: object(7)
memory usage: 282.4+ KB


In [ ]:
df_test_typo= df_test_typo.rename({'label': 'Label', 'Description1':'req1','Description2':'req2'}, axis=1)

In [ ]:
df_test_typo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1130 entries, 29418 to 36787
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          1130 non-null   object
 1   id1                 1130 non-null   object
 2   req1                1130 non-null   object
 3   id2                 1130 non-null   object
 4   req2                1130 non-null   object
 5   Type of dependency  1130 non-null   object
 6   Label               1130 non-null   object
dtypes: object(7)
memory usage: 70.6+ KB


In [ ]:
df_train_typo.to_csv("/content/gdrive/MyDrive/seng607/test_negative_roi/Redmine_train.csv",index=False)

In [ ]:
df_test_typo.to_csv("/content/gdrive/MyDrive/seng607/test_negative_roi/Redmine_test.csv",index=False)


In [ ]:
df_read_train=pd.read_csv("/content/gdrive/MyDrive/seng607/test_negative_roi/Redmine_train.csv")

In [ ]:
print(df_read_train.shape)
print(df_read_train.isnull().sum())

(4496, 7)
Unnamed: 0            0
id1                   0
req1                  0
id2                   0
req2                  0
Type of dependency    0
Label                 0
dtype: int64


In [ ]:
df_read_train.dropna(inplace=True)
df_read_train = df_read_train.reset_index(drop=True)

In [ ]:
df_read_train.to_csv("/content/gdrive/MyDrive/seng607/test_negative_roi/Redmine_train.csv",index=False)

In [ ]:
df_read_train_new=pd.read_csv("/content/gdrive/MyDrive/seng607/test_negative_roi/Redmine_train.csv")

In [ ]:
df_read_train_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4496 entries, 0 to 4495
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          4496 non-null   int64 
 1   id1                 4496 non-null   int64 
 2   req1                4496 non-null   object
 3   id2                 4496 non-null   int64 
 4   req2                4496 non-null   object
 5   Type of dependency  4496 non-null   object
 6   Label               4496 non-null   int64 
dtypes: int64(4), object(3)
memory usage: 246.0+ KB


In [ ]:
df_read_test=pd.read_csv("/content/gdrive/MyDrive/seng607/test_negative_roi/Redmine_test.csv")

In [ ]:
print(df_read_test.shape)
print(df_read_test.isnull().sum())

(1122, 7)
Unnamed: 0            0
id1                   0
req1                  0
id2                   0
req2                  0
Type of dependency    0
Label                 0
dtype: int64


In [ ]:
df_read_test.dropna(inplace=True)
df_read_test = df_read_test.reset_index(drop=True)

In [ ]:
df_read_test.to_csv("/content/gdrive/MyDrive/seng607/test_negative_roi/Redmine_test.csv",index=False)

In [ ]:
df_read_test_new=pd.read_csv("/content/gdrive/MyDrive/seng607/test_negative_roi/Redmine_test.csv")

In [ ]:
df_read_test_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1122 entries, 0 to 1121
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          1122 non-null   int64 
 1   id1                 1122 non-null   int64 
 2   req1                1122 non-null   object
 3   id2                 1122 non-null   int64 
 4   req2                1122 non-null   object
 5   Type of dependency  1122 non-null   object
 6   Label               1122 non-null   int64 
dtypes: int64(4), object(3)
memory usage: 61.5+ KB
